~ Lab Assignment 4

~ Rhythm Rajesh Shah

~ 1233960561

~ 02/22/2025

### Code Cell 1 (5%) - Import all the necessary libraries and restaurant review data.

In [15]:
import json
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import warnings
from keras.layers import TextVectorization
from keras.utils import pad_sequences
import tensorflow as tf
warnings.filterwarnings("ignore")
from tqdm import tqdm

In [7]:
!python3 -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 28.7 MB/s eta 0:00:00 0:00:01

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [8]:
import spacy
nlp = spacy.load("en_core_web_sm")
import nltk

In [12]:
df = pd.read_csv("restaurant_reviews_az.csv")
df.head(5)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,IVS7do_HBzroiCiymNdxDg,fdFgZQQYQJeEAshH4lxSfQ,sGy67CpJctjeCWClWqonjA,3,1,1,0,"OK, the hype about having Hatch chili in your ...",2020-01-27 22:59:06
1,QP2pSzSqpJTMWOCuUuyXkQ,JBLWSXBTKFvJYYiM-FnCOQ,3w7NRntdQ9h0KwDsksIt5Q,5,1,1,1,Pandemic pit stop to have an ice cream.... onl...,2020-04-19 05:33:16
2,oK0cGYStgDOusZKz9B1qug,2_9fKnXChUjC5xArfF8BLg,OMnPtRGmbY8qH_wIILfYKA,5,1,0,0,I was lucky enough to go to the soft opening a...,2020-02-29 19:43:44
3,E_ABvFCNVLbfOgRg3Pv1KQ,9MExTQ76GSKhxSWnTS901g,V9XlikTxq0My4gE8LULsjw,5,0,0,0,I've gone to claim Jumpers all over the US and...,2020-03-14 21:47:07
4,Rd222CrrnXkXukR2iWj69g,LPxuausjvDN88uPr-Q4cQA,CA5BOxKRDPGJgdUQ8OUOpw,4,1,0,0,"If you haven't been to Maynard's kitchen, it'...",2020-01-17 20:32:57


### Code Cell 2 (5%) -  Remove 3-star reviews from the input data and create a new column - Sentiment for the remaining reviews. For reviews with 1 or 2 star ratings, set the value in the Sentiment column to 0. For reviews with 4 or 5-star ratings, set the value in the sentiment column to 1. 

In [14]:
df = df[df['stars'] != 3]

df['Sentiment'] = df['stars'].apply(lambda x: 1 if x > 3 else 0)

df.head(5)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,Sentiment
1,QP2pSzSqpJTMWOCuUuyXkQ,JBLWSXBTKFvJYYiM-FnCOQ,3w7NRntdQ9h0KwDsksIt5Q,5,1,1,1,Pandemic pit stop to have an ice cream.... onl...,2020-04-19 05:33:16,1
2,oK0cGYStgDOusZKz9B1qug,2_9fKnXChUjC5xArfF8BLg,OMnPtRGmbY8qH_wIILfYKA,5,1,0,0,I was lucky enough to go to the soft opening a...,2020-02-29 19:43:44,1
3,E_ABvFCNVLbfOgRg3Pv1KQ,9MExTQ76GSKhxSWnTS901g,V9XlikTxq0My4gE8LULsjw,5,0,0,0,I've gone to claim Jumpers all over the US and...,2020-03-14 21:47:07,1
4,Rd222CrrnXkXukR2iWj69g,LPxuausjvDN88uPr-Q4cQA,CA5BOxKRDPGJgdUQ8OUOpw,4,1,0,0,"If you haven't been to Maynard's kitchen, it'...",2020-01-17 20:32:57,1
5,kx6O_lyLzUnA7Xip5wh2NA,YsINprB2G1DM8qG1hbrPUg,rViAhfKLKmwbhTKROM9m0w,1,0,0,0,I stay at the Main Hotel at the Casino from Ju...,2020-07-14 16:43:23,0


### Code Cell 3 (10%) - Conduct necessary data processing. Prepare the training and test sets on review data for machine learning classifications. 20% of the data is for testing, and 80% of the data is for training. 

In [19]:
def normalize_single_review(doc, lowercase=True, remove_stopwords=True):
    lemmatized = []
    for token in doc:
        lemma = token.lemma_.lower() if lowercase else token.lemma_
        if not remove_stopwords or (remove_stopwords and not token.is_stop):
            lemmatized.append(lemma)
    return " ".join(lemmatized)

# Process the dataset in chunks
def process_in_chunks(df, chunk_size=1000, batch_size=500, lowercase=True, remove_stopwords=True):
    results = []

    for i in tqdm(range(0, len(df), chunk_size)):  # Loop through the dataset in chunks
        chunk = df['text'].iloc[i:i+chunk_size].tolist()

        # Process each chunk using spaCy's pipe for efficiency
        processed_chunk = []
        for doc in nlp.pipe(chunk, batch_size=batch_size, n_process=1):  # Use a limited number of processes
            processed_chunk.append(normalize_single_review(doc, lowercase=lowercase, remove_stopwords=remove_stopwords))

        results.extend(processed_chunk)  # Add the processed chunk to results

    return results

In [20]:
#methods from nltk
# to remove non alphanumeric character
def alpha_num(text):
    return re.sub(r'[^A-Za-z0-9 ]', '', text)

# to remove URLs
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

# to remove html tags
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

In [21]:
# apply preprocessing steps
df['text'] = df['text'].apply(remove_URL)
print("remove_URL function done")
df['text'] = df['text'].apply(remove_html)
print("remove_html function done")
df['text'] = df['text'].apply(alpha_num)
print("alpha_num function done")
# Apply the chunked processing to the entire dataframe
# df['review'] = df['review'].apply(normalize,lowercase=True, remove_stopwords=True)
df['preview'] = process_in_chunks(df, chunk_size=1000, batch_size=500, lowercase=True, remove_stopwords=True)

# Display the processed dataframe
df.head()

remove_URL function done
remove_html function done
alpha_num function done


100%|██████████| 45/45 [06:34<00:00,  8.77s/it]


,review_id,user_id,business_id,stars,useful,funny,cool,text,date,Sentiment,preview
1,QP2pSzSqpJTMWOCuUuyXkQ,JBLWSXBTKFvJYYiM-FnCOQ,3w7NRntdQ9h0KwDsksIt5Q,5,1,1,1,Pandemic pit stop to have an ice cream only pl...,2020-04-19 05:33:16,1,pandemic pit stop ice cream plain sundae limit...
2,oK0cGYStgDOusZKz9B1qug,2_9fKnXChUjC5xArfF8BLg,OMnPtRGmbY8qH_wIILfYKA,5,1,0,0,I was lucky enough to go to the soft opening a...,2020-02-29 19:43:44,1,lucky soft opening let tell good beer wine dif...
3,E_ABvFCNVLbfOgRg3Pv1KQ,9MExTQ76GSKhxSWnTS901g,V9XlikTxq0My4gE8LULsjw,5,0,0,0,Ive gone to claim Jumpers all over the US and ...,2020-03-14 21:47:07,1,ve go claim jumper disappoint location differe...
4,Rd222CrrnXkXukR2iWj69g,LPxuausjvDN88uPr-Q4cQA,CA5BOxKRDPGJgdUQ8OUOpw,4,1,0,0,If you havent been to Maynards kitchen its ti...,2020-01-17 20:32:57,1,not maynards kitchen time hope dinner sure r...
5,kx6O_lyLzUnA7Xip5wh2NA,YsINprB2G1DM8qG1hbrPUg,rViAhfKLKmwbhTKROM9m0w,1,0,0,0,I stay at the Main Hotel at the Casino from Ju...,2020-07-14 16:43:23,0,stay main hotel casino july 11 july 13 bad exp...


In [23]:
train_sentences, test_sentences, train_labels, test_labels = train_test_split(df['preview'], df['Sentiment'], test_size=0.2, random_state=5, stratify=df['Sentiment'])

### Code Cell 4 (10%) - Download the pre-trained GloVe word embeddings (glove.6B.100d.txt) and prepare the embedding matrix for this review dataset.